Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Install and Import Libraries
Let's install PySpark:

In [1]:
import findspark
findspark.init()


In [2]:
import pyspark
from pyspark.sql import SparkSession

## Build Spark Session

In [3]:
spark = SparkSession.builder.master('local[*]').appName("OmarSession").getOrCreate()

## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



In [4]:
df_train = spark.read.csv("train.csv" , inferSchema=True ,header=True)
df_test = spark.read.csv("test.csv" , inferSchema=True ,header=True)

Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [5]:
type(df_train)

pyspark.sql.dataframe.DataFrame

**Show 5 rows.**

In [8]:
df_train.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

**Display schema for the dataset:**

In [9]:
df_train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [22]:

df_train.summary().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [18]:
df_train.count()

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [29]:
from pyspark.sql.functions import sum,avg
survive_df=df_train.groupBy('Survived').count()


0


**Display your result:**

In [27]:
survive_df.show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



**Can you display your answer in ratio form?(Hint: Use "UDF" Function. (Hint: Use "UDF" Function. This is a hint you can use any method.)**






In [42]:
from pyspark.sql.functions import udf,col
total=df_train.count()
ratio_udf=udf(lambda x:x/total)
survive_df.withColumn("Survived",ratio_udf(survive_df.Survived)).show()

+--------------------+-----+
|            Survived|count|
+--------------------+-----+
|0.001122334455667789|  342|
|                 0.0|  549|
+--------------------+-----+



**Can you get the number of males and females?**


In [43]:
sex_df=df_train.groupBy('Sex').count()
sex_df.show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+



**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column. This is a hint you can use any method.)

In [63]:
import pyspark.sql.functions as sqlFuncs
sex_avg_df=df_train.groupby('Sex').agg(sqlFuncs.avg(sqlFuncs.col('Survived'))).show()




+------+-------------------+
|   Sex|      avg(Survived)|
+------+-------------------+
|female| 0.7420382165605095|
|  male|0.18890814558058924|
+------+-------------------+



**Create temporary view PySpark:**

In [65]:
df_train.createOrReplaceTempView("tempView")

**How many people survived, and how many didn't survive? By SQL:**

In [72]:
spark.sql('Select Survived,count(Survived) from tempView \
          group by Survived  \
          ').show()

+--------+---------------+
|Survived|count(Survived)|
+--------+---------------+
|       1|            342|
|       0|            549|
+--------+---------------+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column. This is a hint you can use any method.)

**Can you do this via SQL?**

In [73]:
spark.sql('Select Sex,avg(Survived) from tempView \
          group by Sex  \
          ').show()

+------+-------------------+
|   Sex|      avg(Survived)|
+------+-------------------+
|female| 0.7420382165605095|
|  male|0.18890814558058924|
+------+-------------------+



**Display a ratio for "p-class": SUM(Survived)/count for p-class**


In [94]:
spark.sql('Select pclass,sum(Survived)/count(pclass) as ratio  from tempView  group by pclass').show()

+------+-------------------+
|pclass|              ratio|
+------+-------------------+
|     1| 0.6296296296296297|
|     3|0.24236252545824846|
|     2|0.47282608695652173|
+------+-------------------+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [81]:
df_union=df_train.union(df_test)

**Display count:**

In [82]:
df_union.count()

1329

**Can you define the number of null values in each column?**


In [90]:
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
df_union.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_union.columns]
   ).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



**Create Dataframe for null values**

1. Column
2. Number of missing values.

## Preprocessing 

**Create Temporary view PySpark:**

In [103]:
df_union.createOrReplaceTempView("PySpark")

**Can you show the "name" column from your temporary table?**

In [104]:
spark.sql("Select name from PySpark").show()

+--------------------+
|                name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
|Sandstrom, Miss. ...|
|Bonnell, Miss. El...|
|Saundercock, Mr. ...|
|Andersson, Mr. An...|
|Vestrom, Miss. Hu...|
|Hewlett, Mrs. (Ma...|
|Rice, Master. Eugene|
|Williams, Mr. Cha...|
|Vander Planke, Mr...|
|Masselmani, Mrs. ...|
+--------------------+
only showing top 20 rows



**Run this code:**

In [133]:
import pyspark.sql.functions as F
combined = df_union.withColumn('Title',F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))
combined.createOrReplaceTempView('PySpark')

**Display "Title" column and count "Title" column:**

In [138]:

spark.sql('Select Title,count(Title) \
 from PySpark  group by Title').show()

+--------+------------+
|   Title|count(Title)|
+--------+------------+
|     Don|           1|
|    Miss|         257|
|Countess|           2|
|     Col|           4|
|     Rev|           9|
|    Lady|           2|
|  Master|          56|
|     Mme|           1|
|    Capt|           2|
|      Mr|         786|
|      Dr|          11|
|     Mrs|         186|
|     Sir|           2|
|Jonkheer|           2|
|    Mlle|           4|
|   Major|           3|
|      Ms|           1|
+--------+------------+



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [152]:
rare_list = ["Dr", "Rev", "Major", "Col", "Mlle", "Capt", "Don", "Jonkheer", "Countess", "Ms", "Sir", "Lady", "Mme"]
titles_map = {
 'Capt': 'Rare',
 'Col': 'Rare',
 'Don': 'Rare',
 'Dona': 'Rare',
 'Dr': 'Rare',
 'Jonkheer':'Rare' ,
 'Lady': 'Rare',
 'Major': 'Rare',
 'Master': 'Master',
 'Miss': 'Miss',
 'Mlle': 'Rare',
 'Mme': 'Rare',
 'Mr': 'Mr',
 'Mrs': 'Mrs',
 'Ms': 'Rare',
 'Rev': 'Rare',
 'Sir': 'Rare',
 'Countess': 'Rare'
}

**Run the function:**

In [153]:
def impute_title(title):
    return titles_map[title]# Title_map is your dictionary. please change this name with your dictionary name.

**Apply the function on "Title" column using UDF:**

In [154]:
from pyspark.sql.types import *
def set_title(title):
    if title in rare_list:
        return "rare"
    else:
        return title
    
udf_data = udf(lambda z: impute_title(z),StringType())

title_df = combined.withColumn("Title",udf_data(combined['Title']))


**Display "Title" from table and group by "Title" column:**

In [151]:
title_df.groupBy("Title").count().show()

+------+-----+
| Title|count|
+------+-----+
|  Miss|  257|
|Master|   56|
|    Mr|  786|
|   Mrs|  186|
|  Rare|   44|
+------+-----+



## **Preprocessing Age**

**Based on the "age" column mean, you will fill in the missing age values:**

In [189]:
avg_age = combined.agg({"age": "avg"}).collect()
avg_age[0][0]

30.079501879699244

In [190]:
combined.na.fill({'age': avg}).show()

combined = combined.na.fill(avg_age[0][0],subset = ['Age'])
combined.show(truncate= False)

+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|              22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|              38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|              26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|              35.0|    1|    0|          113803|   53.1| C123|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia..

**Fill missing with "age" mean:**

## **Preprocessing Embarked**

**Select "Embarked" column, count them, order by count Desc, and save in grouped_Embarked variable:**




In [198]:
grouped_Embarked=combined.groupBy('Embarked').count().orderBy(col('count').desc())

**Show "groupped_Embarked" your variable:**

In [199]:
grouped_Embarked.show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  962|
|       C|  253|
|       Q|  111|
|    null|    3|
+--------+-----+



**Get max of groupped_Embarked:** 

In [201]:
grouped_Embarked.show(1)

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  962|
+--------+-----+
only showing top 1 row



**Fill missing values with max 'S' of grouped_Embarked:**

In [202]:
combined = combined.na.fill('S',subset = ['Embarked'])
combined.show(truncate= False)

+-----------+--------+------+-------------------------------------------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|Name                                                   |Sex   |Age               |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|Title |
+-----------+--------+------+-------------------------------------------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|1          |0       |3     |Braund, Mr. Owen Harris                                |male  |22.0              |1    |0    |A/5 21171       |7.25   |null |S       |Mr    |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)    |female|38.0              |1    |0    |PC 17599        |71.2833|C85  |C       |Mrs   |
|3          |1       |3     |Heikkinen, Miss. Laina                                 |female|26.0              |0    |0    |STON/O2. 3101282|7.925

## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



In [203]:
from pyspark.sql.functions import substring
combined = combined.withColumn('Cabin', substring('Cabin', 1, 1))

**Show the result:**

In [205]:
combined.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Title|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|   Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|    C|       C|  Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S| Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|    C|       S|  Mrs|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|   Mr|
+-----------+--------+------+---

**Create the temporary view:**

In [206]:
combined.createOrReplaceTempView('PySpark')

**Select "Cabin" column, count "Cabin" column, Group by "Cabin" column, Order By count DESC**  

In [210]:
spark.sql('Select Cabin,count(Cabin) as count from PySpark group by Cabin order by count DESC').show()

+-----+-----+
|Cabin|count|
+-----+-----+
|    C|   82|
|    B|   77|
|    D|   52|
|    E|   51|
|    A|   23|
|    F|   18|
|    G|    4|
|    T|    1|
| null|    0|
+-----+-----+



**Fill missing values with "U":**

In [212]:
combined = combined.na.fill('U',subset = ['Cabin'])
combined.show(5,truncate= False)

+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|PassengerId|Survived|Pclass|Name                                               |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|Title|
+-----------+--------+------+---------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|1          |0       |3     |Braund, Mr. Owen Harris                            |male  |22.0|1    |0    |A/5 21171       |7.25   |U    |S       |Mr   |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)|female|38.0|1    |0    |PC 17599        |71.2833|C    |C       |Mrs  |
|3          |1       |3     |Heikkinen, Miss. Laina                             |female|26.0|0    |0    |STON/O2. 3101282|7.925  |U    |S       |Miss |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)       |female|

**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = false)
 |-- Embarked: string (nullable = false)
 |-- Title: string (nullable = true)



**OneHotEncoder(inputCols=None, outputCols=None)**

A one-hot encoder that maps a column of category indices to a column of binary vectors, with at most a single one-value per row that indicates the input category index. For example with 5 categories, an input value of 2.0 would map to an output vector of [0.0, 0.0, 1.0, 0.0]. The last category is not included by default (configurable via dropLast), because it makes the vector entries sum up to one, and hence linearly dependent. So an input value of 4.0 maps to [0.0, 0.0, 0.0, 0.0].

In [216]:
categoricalCols = ["Sex", "Embarked", "Title", "Cabin"]
indexOutputCols = [x + "_Index" for x in categoricalCols]
indexOutputCols

from pyspark.ml.feature import StringIndexer

stringIndexer = StringIndexer(inputCols=categoricalCols,
                             outputCols=indexOutputCols,
                             handleInvalid='skip')

In [217]:
numericCols=['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

assemblerInputs = indexOutputCols + numericCols
assemblerInputs

['Sex_Index',
 'Embarked_Index',
 'Title_Index',
 'Cabin_Index',
 'Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare']

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None). A feature transformer that merges multiple columns into a vector column.**



In [218]:
from pyspark.ml.feature import VectorAssembler
vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')

**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [219]:
# split data
x_train,X_test=combined.randomSplit([.8,.2],seed=42)

**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [224]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol='Survived',featuresCol='features')
from pyspark.ml import Pipeline
pipeline =Pipeline(stages = [stringIndexer,vecAssembler,rf])

pipelineModel = pipeline.fit(x_train)
predDF = pipelineModel.transform(X_test)



**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [225]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
classificationEvaluator = MulticlassClassificationEvaluator(predictionCol='prediction',
                                         labelCol='Survived',
                                         metricName='accuracy')

accuracy = classificationEvaluator.evaluate(predDF)

print(accuracy)

0.8418803418803419


**When you are finished send the project via Google classroom**
**Please let me know if you have any questions.**
* nabieh.mostafa@yahoo.com
* +201015197566 (Whatsapp)

**Don't Hate me, I push you to learn**

**I will help you to become an awesome data engineer.**

**Why did I say that "Data Engineer"?**

**Tricky question, but an optional question, if you would like to know the answer, ask me.**
